In [7]:
!git clone https://github.com/lisapraino/Deep_learning.git

Cloning into 'Deep_learning'...
remote: Enumerating objects: 5011, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5011 (delta 0), reused 1 (delta 0), pack-reused 5009 (from 2)
Receiving objects: 100% (5011/5011), 371.38 MiB | 15.83 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (6300/6300), done.


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s eta 0:00:00


In [8]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")
model.train(data="/alternative-dataset/Fridge_detection.v1i.yolov11/data.yaml", epochs=25, batch=8, lrf=10**(-3))

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/alternative-dataset/Fridge_detection.v1i.yolov11/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, per

RuntimeError: Dataset '/alternative-dataset/Fridge_detection.v1i.yolov11/data.yaml' error ❌ '/alternative-dataset/Fridge_detection.v1i.yolov11/data.yaml' does not exist

In [ ]:
result1 = model("/alternative-dataset/Fridge_detection.v1i.yolov11/test/images/DSC_5680_JPG_jpg.rf.f7db913a7619d626d898768562ffd67a.jpg")
result1 = result1[0]
result1.show()